# SCC5977 - Aprendizado de Máquina para Séries Temporais (2024)

## Grupo
> André Guarnier De Mitri - 11395579
> 
> 

## Problema


In [27]:
# Importing necessary libraries
from aeon.distances import pairwise_distance
from aeon.distances import ddtw_distance
import numpy as np
from aeon.datasets import load_from_tsfile

# Load time series data
X_train, y_train = load_from_tsfile(full_file_path_and_name='./data/ts_files/train.ts')
X_test, y_test = load_from_tsfile(full_file_path_and_name='./data/ts_files/test.ts')

# Validate loaded data
print(f"Loaded {X_train.shape[0]} training samples with shape {X_train.shape[1:]} and {X_test.shape[0]} test samples.")

# Selecting the first instance from train and test for demonstration purposes
time_series_1 = X_train[0, 0]  # First time series from the training set
time_series_2 = X_test[0, 0]  # First time series from the test set

# Check if the selected data is valid
euclidean_dist = pairwise_distance(time_series_1, time_series_2, metric="euclidean")

# Calculate Derivative Dynamic Time Warping distance
ddtw_dist = ddtw_distance(time_series_1, time_series_2)

# Print results
print(f"Euclidean Distance: {euclidean_dist}")
print(f"Derivative DTW Distance: {ddtw_dist}")

Loaded 6000 training samples with shape (1, 137) and 1500 test samples.
Euclidean Distance: [[2.02366115]]
Derivative DTW Distance: 0.3739516388002304


# Trabalhos relacionados

# Algoritimos Utilizados

# Metodologia

## Carregando o dataset

# Resultados e Discussões